In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator
from qiskit.circuit.library import QFT
import numpy as np

# Use Aer simulator backend
backend = Aer.get_backend('qasm_simulator')

In [4]:

# Create the quantum circuit with 3 qubits and 3 classical bits
q = QuantumRegister(3, 'q')  # Quantum register
c0 = ClassicalRegister(1, 'c0')  # Classical register for Alice's qubit
c1 = ClassicalRegister(1, 'c1')  # Classical register for Bob's qubit
c2 = ClassicalRegister(1, 'c2')  # Classical register for the result
circuit = QuantumCircuit(q, c0, c1, c2)

# Prepare the initial state to be teleported
circuit.initialize([0, 1], q[0])  # Initialize q[0] in the |1> state
circuit.barrier()

# Create an entanglement between Alice's and Bob's qubits
circuit.h(q[1])  # Hadamard on Bob's qubit
circuit.cx(q[1], q[2])  # CNOT between Bob's qubits
circuit.barrier()

# Teleportation process
circuit.cx(q[0], q[1])  # CNOT for teleportation
circuit.h(q[0])  # Hadamard on Alice's qubit
circuit.barrier()

# Measure Alice's qubits and send the measurement results to Bob
circuit.measure(q[0], c0[0])  # Measure Alice's qubit
circuit.measure(q[1], c1[0])  # Measure Bob's qubit

# Apply corrective operations on Bob's qubit based on the measurement results
circuit.x(q[2]).c_if(c1, 1)  # Apply X gate if Bob's measurement is 1
circuit.z(q[2]).c_if(c0, 1)  # Apply Z gate if Alice's measurement is 1

# Measure the teleported qubit
circuit.measure(q[2], c2[0])  # Measure the teleported state

# Visualize the circuit
print(circuit)
circuit.draw(output='mpl')

# Simulate the circuit using the QASM simulator
transpiled_circuit = transpile(circuit, backend)  # Transpile the circuit for the simulator
job = backend.run(transpiled_circuit, shots=1)  # Run the transpiled circuit
result = job.result()
teleported_state = result.get_counts(transpiled_circuit)  # Get counts from the transpiled circuit

# Print the teleported state
print("Teleported state:", teleported_state)

      ┌─────────────────┐ ░            ░      ┌───┐ ░ ┌─┐                    
 q_0: ┤ Initialize(0,1) ├─░────────────░───■──┤ H ├─░─┤M├────────────────────
      └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐                 
 q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─────────────────
                          ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ┌───┐  ┌───┐ ┌─┐
 q_2: ────────────────────░──────┤ X ├─░────────────░──╫──╫──┤ X ├──┤ Z ├─┤M├
                          ░      └───┘ ░            ░  ║  ║  └─╥─┘  └─╥─┘ └╥┘
                                                       ║  ║    ║   ┌──╨──┐ ║ 
c0: 1/═════════════════════════════════════════════════╩══╬════╬═══╡ 0x1 ╞═╬═
                                                       0  ║ ┌──╨──┐└─────┘ ║ 
c1: 1/════════════════════════════════════════════════════╩═╡ 0x1 ╞════════╬═
                                                          0 └─────┘        ║ 
c2: 1/══════════════════════════════════════════════════════════